<a href="https://colab.research.google.com/github/Linthevanrooij/FAIP/blob/main/Final_project/Text%2Bimage-generator/llama2-generator/llama2_sequence_prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Llama 2

In [1]:
!pip install transformers
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
from transformers import AutoTokenizer
import transformers
import torch
import json

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [3]:
with open('all_scores.json', 'r') as file:
    data = json.load(file)

In [4]:
def extra_emotion(valence, arousal):
  emotion = f"""
  Take into account my current emotional state without explicitly mentioning it. Valence is ranging from negative to positive. Arousal is intensity of emotion. Both ranging from 1-9.
  Arousal: {arousal}
  Valence: {valence}
  """
  return emotion

In [40]:
def do_llama_seq(prompt):
  sequences = pipeline(
    prompt,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens = 1000,
    return_full_text=False
    )
  return sequences[0]['generated_text']

In [41]:
def make_texts(stage_num, data):
    stage = data[f'stage_{stage_num}']
    print(f"Generating texts for stage{stage_num}")

    prompt_baseline = stage['prompt']
    prompt_positive = stage['prompt'] + " " + extra_emotion(stage['valence_pos'], stage['arousal_pos'])
    prompt_negative = stage['prompt'] + " " + extra_emotion(stage['valence_neg'], stage['arousal_neg'])

    print('Generating baseline')
    text_baseline = do_llama_seq(prompt_baseline)
    print('Generating positive')
    text_positive = do_llama_seq(prompt_positive)
    print('Generating negative')
    text_negative = do_llama_seq(prompt_negative)

    data[f'stage_{stage_num}']['text_bas'] = text_baseline
    data[f'stage_{stage_num}']['text_pos'] = text_positive
    data[f'stage_{stage_num}']['text_neg'] = text_negative



In [42]:
for i in range(1,2):
  make_texts(i, data)

Generating texts for stage1
Generating baseline
Generating positive
Generating negative


In [16]:
texts = json.dumps(data, indent=2)
with open('text_and_scores.json', 'w') as f:
  f.write(texts)

In [22]:
with open('text_and_scores.json', 'r') as f:
  data_all = json.load(f)

In [27]:
print(data_all['stage_1']['text_neg'])

Help me define a design brief for a modern chair. The target audience is young professionals looking for a minimalist, ergonomic chair for home offices. Include materials and aesthetics. 
  Take into account my current emotional state without explicitly mentioning it. Valence is ranging from negative to positive. Arousal is intensity of emotion. Both ranging from 1-9.
  Arousal: 7.35
  Valence: 1.84
  
  I want to create a design brief for a modern chair that is minimalist, ergonomic, and comfortable. The target audience is young professionals who work from home and are looking for a chair that can help them stay focused and productive throughout the day. The chair should be made of high-quality materials and have a sleek, modern aesthetic that complements any home office setup.

  The design should prioritize comfort and ergonomics, with features such as adjustable height and tilt, lumbar support, and a breathable mesh fabric. The chair should also be easy to assemble and disassemble,